In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install pandas openpyxl

In [ ]:
# https://console.cloud.google.com/apis/credentials/key/1dd740a7-bc82-47cf-ba84-b53581abd56b?hl=ko&project=prefab-bedrock-423206-m2 유튜브 API 이용

from googleapiclient.discovery import build

# API 키를 입력
api_key = 'API_KEY'

youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
# # top-level comments만 크롤링(대댓글 X)
# # 시간대별로
# # 검색어 - 카리나 이재욱
# # https://www.youtube.com/watch?v=6Xy3BNRswJ8 에스파 카리나·이재욱 열애 인정…"알아가는 중" / KNN

# import pandas as pd

# def get_all_comments(video_id, published_after, published_before):
#     comments = []
#     next_page_token = None

#     while True:
#         response = youtube.commentThreads().list(
#             part="snippet",
#             videoId=video_id,
#             textFormat="plainText",
#             maxResults=100,
#             pageToken=next_page_token
#         ).execute()

#         for item in response['items']:
#             comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
#             author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
#             published_at = item['snippet']['topLevelComment']['snippet']['publishedAt']
#             like_count = item['snippet']['topLevelComment']['snippet']['likeCount']

#             if published_after <= published_at.split("T")[0] <= published_before:
#                  comments.append({
#                 'Author': author,
#                 'Comment': comment,
#                 'Published At': published_at,
#                 'Like Count': like_count
#             })

#         next_page_token = response.get('nextPageToken')
#         if not next_page_token:
#             break

#     return comments

# def save_comments_to_excel(comments, filename):
#     # 데이터프레임 생성
#     df = pd.DataFrame(comments)
#     # 엑셀 파일로 저장
#     df.to_excel(filename, index=False)

# # 동영상 ID 입력
# video_id = '6Xy3BNRswJ8'
# # 특정 날짜 범위 설정 (예: 2023-06-01 ~ 2023-06-02)
# published_after = "2023-06-01"
# published_before = "2023-06-02"
# comments = get_all_comments(video_id, published_after, published_before)
# save_comments_to_excel(comments, 'YouTube_Comments.xlsx')
# print("Comments have been saved to Excel.")

In [ ]:
# top-level comments만 크롤링(대댓글 X)
# 검색어 - 카리나 이재욱
# https://www.youtube.com/watch?v=6Xy3BNRswJ8 에스파 카리나·이재욱 열애 인정…"알아가는 중" / KNN

import pandas as pd

def get_all_comments(video_id):
    comments = []
    next_page_token = None

    while True:
        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        ).execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            published_at = item['snippet']['topLevelComment']['snippet']['publishedAt']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
            comments.append({
                'Author': author,
                'Comment': comment,
                'Published At': published_at,
                'Like Count': like_count
            })

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return comments

def save_comments_to_excel(comments, filename):
    # 데이터프레임 생성
    df = pd.DataFrame(comments)
    # 엑셀 파일로 저장
    df.to_excel(filename, index=False)

# 동영상 ID 입력
video_id = '6Xy3BNRswJ8'
comments = get_all_comments(video_id)
save_comments_to_excel(comments, 'YouTube_Comments.xlsx')
print("Comments have been saved to Excel.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # 대댓글까지 포함해서 크롤링
# import pandas as pd

# def get_all_comments(video_id):
#     comments = []
#     next_page_token = None

#     while True:
#         response = youtube.commentThreads().list(
#             part="snippet,replies",  # replies 부분 추가
#             videoId=video_id,
#             textFormat="plainText",
#             maxResults=100,
#             pageToken=next_page_token
#         ).execute()

#         for item in response['items']:
#             top_level_comment = item['snippet']['topLevelComment']
#             author = top_level_comment['snippet']['authorDisplayName']
#             comment = top_level_comment['snippet']['textDisplay']
#             published_at = top_level_comment['snippet']['publishedAt']
#             like_count = top_level_comment['snippet']['likeCount']
#             comments.append({
#                 'Author': author,
#                 'Comment': comment,
#                 'Published At': published_at,
#                 'Like Count': like_count
#             })
#             # 답변 댓글 처리
#             if 'replies' in item:
#                 for reply in item['replies']['comments']:
#                     r_author = reply['snippet']['authorDisplayName']
#                     r_comment = reply['snippet']['textDisplay']
#                     r_published_at = reply['snippet']['publishedAt']
#                     r_like_count = reply['snippet']['likeCount']
#                     comments.append({
#                         'Author': r_author,
#                         'Comment': r_comment,
#                         'Published At': r_published_at,
#                         'Like Count': r_like_count
#                     })

#         next_page_token = response.get('nextPageToken')
#         if not next_page_token:
#             break

#     return comments

# def save_comments_to_excel(comments, filename):
#     # 데이터프레임 생성
#     df = pd.DataFrame(comments)
#     # 엑셀 파일로 저장
#     df.to_excel(filename, index=False)

# # 동영상 ID 입력
# video_id = '6Xy3BNRswJ8'
# comments = get_all_comments(video_id)
# save_comments_to_excel(comments, 'YouTube_Comments.xlsx')
# print("Comments have been saved to Excel.")

In [ ]:
# 추가로 크롤링해서 기존 파일에 이어붙이기

import pandas as pd

# 기존 엑셀 파일 불러오기
def load_existing_data(filename):
    try:
        return pd.read_excel(filename)
    except FileNotFoundError:
        # 파일이 존재하지 않을 경우 빈 데이터프레임 반환
        return pd.DataFrame()

# 새 댓글 데이터를 엑셀 파일에 추가
def append_comments_to_excel(new_comments, filename):
    # 기존 데이터 불러오기
    existing_data = load_existing_data(filename)

    # 새 데이터를 DataFrame으로 변환
    new_data = pd.DataFrame(new_comments)

    # 기존 데이터에 새 데이터 추가
    updated_data = pd.concat([existing_data, new_data], ignore_index=True)

    # 엑셀 파일로 저장
    updated_data.to_excel(filename, index=False)

# 예시 사용법
# 새로 수집한 댓글 데이터 (예: new_comments) 이 여기에 있어야 함
# video_id = 'NEW_VIDEO_ID'
# new_comments = get_all_comments(video_id)
# append_comments_to_excel(new_comments, 'YouTube_Comments.xlsx')

In [ ]:
# 동영상 ID 입력
# https://www.youtube.com/watch?v=VLPvBKZRv9g [문화연예 플러스] '에스파' 카리나·배우 이재욱, 열애 인정 (2024.02.28/뉴스투데이/MBC)
# https://www.youtube.com/watch?v=uRcTvzQ7Fw0 에스파 카리나-배우 이재욱 열애 인정..."잘 어울린다"
# https://www.youtube.com/watch?v=zIX8IUzslCo 에스파 카리나[aespa Karina], 배우 이재욱과 첫 눈에 반했다?!(ENG, CHN, JPN)
# https://www.youtube.com/watch?v=xzIxhCuMG-s 카리나 이재욱 열애 인정 후 나오는 말들
# https://www.youtube.com/watch?v=2pNqjaSpw30 배우 이재욱ㆍ에스파 카리나 열애 인정…"알아가는 단계" / 연합뉴스TV (YonhapnewsTV)
# https://www.youtube.com/watch?v=jQop_sKJLxM 이재욱·카리나 열애…"첫 만남 순간부터 사랑에 빠져" / SBS / 굿모닝연예
# https://www.youtube.com/watch?v=cGv1NVS0HI8 에스파 카리나 연애 소식 팬들 떠나간다ㄷㄷ
# https://www.youtube.com/watch?v=3hLxL3qq5x8 카리나 이재욱 열애인정
# https://www.youtube.com/watch?v=uTiGtPBqUbY [Y이슈] Z세대 아이돌 방식? 카리나♥이재욱 열애에 연예부 기자도 놀란 이유/ YTN star
# https://www.youtube.com/watch?v=y8OIG3zO41c 에스파 카리나·배우 이재욱 공개 열애…첫 만남 재조명 [잇슈 연예] / KBS 2024.02.28.
# https://www.youtube.com/watch?v=0sKn1UjntqM 에스파 카리나 이재욱과 연애 소식 나왔다ㄷㄷ
# https://www.youtube.com/watch?v=ao8LyvKYepM 카리나 이재욱 열애설로 팬덤 분열된 진짜 이유
# https://www.youtube.com/watch?v=zpBM_ilmvo4 [한밤X모닝와이드]공식 열애 인정!! 카리나❤️이재욱
# https://www.youtube.com/watch?v=I8S5L_3icuo 카리나 열애설에 거금 쏟은 팬들 격분 "안티팬 되는 건 한순간"…열애 인정 후폭풍 #karina #aespa
# https://www.youtube.com/watch?v=LTFvGXhzd8Q 생각보다 분위기 안좋은 에스파 공식계정 카리나 팬들 반응 #aespa #karina


# https://www.youtube.com/shorts/Pc62AMY_WQ8 첫만남부터 심상치 않았던 두 사람❤카리나 이재욱 열애
# https://www.youtube.com/shorts/GOgbYUq0Q_c 카리나 이재욱 열애설 디스패치
# https://www.youtube.com/shorts/DinVQ48oLZs 카리나♥️이재욱 열애설에 의외의 반응
# https://www.youtube.com/shorts/BuMLghv44Jk 이제는 보인다는 '카리나♥️이재욱' 시그널
# https://www.youtube.com/shorts/_DnQygvnJTw 카리나 이재욱 열애설, 의문점과 팬들 반응
# https://www.youtube.com/shorts/aqSkAaPpfmc 카리나 이재욱 열애, 이탈리아에서...
# https://www.youtube.com/shorts/WcyfR3nqV0U 카리나 이재욱 초고속 열애 인정ㄷㄷ
# https://www.youtube.com/shorts/EZENNMmxwPU 카리나♥️이재욱, 어떻게 만났을까?
# https://www.youtube.com/shorts/kY8HakTyZdo 카리나 보고 첫눈에 반한 이재욱 (열애설 인정)
# https://www.youtube.com/shorts/e4mVwvaiL-s 소름ㄷㄷ 이미 예견되어있었다는 카리나💗이재욱 공개연애
# https://www.youtube.com/shorts/eVltJSaCnRY 카리나 이재욱 연애, 돈쓰는 코어팬들 탈덕 찐반응



video_ids = ['VLPvBKZRv9g', 'uRcTvzQ7Fw0', 'zIX8IUzslCo', 'xzIxhCuMG-s', '2pNqjaSpw30', 'jQop_sKJLxM', 'cGv1NVS0HI8', '3hLxL3qq5x8', 'uTiGtPBqUbY', 'y8OIG3zO41c', '0sKn1UjntqM', 'ao8LyvKYepM',
             'zpBM_ilmvo4', 'I8S5L_3icuo', 'LTFvGXhzd8Q', 'Pc62AMY_WQ8', 'GOgbYUq0Q_c', 'DinVQ48oLZs', 'BuMLghv44Jk', '_DnQygvnJTw', 'aqSkAaPpfmc', 'WcyfR3nqV0U', 'EZENNMmxwPU', 'kY8HakTyZdo',
             'e4mVwvaiL-s', 'eVltJSaCnRY']

for video_id in video_ids:
    new_comments = get_all_comments(video_id)
    append_comments_to_excel(new_comments, 'YouTube_Comments.xlsx')
    print(f"Comments for video ID {video_id} have been appended to the existing Excel file.")
print("All new comments have been appended to the existing Excel file.")